In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import os
import numpy as np
from emoji import core
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from urllib.parse import quote_plus          
import time
from pathlib import Path
import os
from urllib.request import (urlopen, urlparse, urlunparse, urlretrieve)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import math
import re
from selenium.webdriver.chrome.service import Service
import os 
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import pdb
import os
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import traceback         
from selenium.webdriver.common.proxy import Proxy, ProxyType
import csv
import requests
import json
import random
import psutil
from datetime import datetime
import pickle
import warnings

In [2]:

def selenium_scroll_down(driver):
    SCROLL_PAUSE_SEC = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        new_height = driver.execute_script("return document.body.scrollHeight")
  
        if new_height == last_height:
            return 1
        last_height = new_height


def get_driver(chrome_path, chrome_options, url):
    driver = None
    count = 0
    
    while (driver == None) and (count < 10):
            try:
                driver = webdriver.Chrome(chrome_path, options=chrome_options)
            except Exception:
                count = count + 1
                clean_up()
                if driver: driver.quit()
                continue
    connect = False
    
    while connect == False: 
        try:
            driver.get(url)
            driver.implicitly_wait(10)
            connect = True
        except Exception:
            del driver
            driver = webdriver.Chrome(chrome_path, options=chrome_options)
            continue
    return driver

def reset_driver(driver, chrome_path, chrome_options, url, cookies):

    try :
        driver.quit()
        driver = get_driver(chrome_path, chrome_options, url, cookies)
    except Exception:
        driver = get_driver(chrome_path, chrome_options, url, cookies)
    return driver


def write_data(write_file, datas):

    for data in datas:
        write_file = write_file.append(data, ignore_index = True)
    
    return write_file



In [3]:
def get_name(cafe):
    try:
        return cafe.find_element(By.CLASS_NAME, 'link_name').text
    except Exception as e: 
        print(e)
        return None
    
def get_category(cafe):
    try:
        return cafe.find_element(By.CLASS_NAME, 'subcategory.clickable').text
    except Exception as e: 
        print(e)
        return None
def get_loc(cafe):
    try:
        return cafe.find_element(By.CLASS_NAME, 'addr').text.split('\n')[0]
    except Exception as e: 
        print(e)
        return None
def get_link(cafe):
    try:
        link_area = cafe.find_element(By.CLASS_NAME, 'contact.clickArea')
        return link_area.find_element(By.CLASS_NAME, 'moreview').get_attribute('href')
    except Exception as e: 
        print(e)
        return None
def get_rating_info(cafe):
    try:
        rating_area = WebDriverWait(cafe, 8).until(EC.presence_of_element_located((By.CLASS_NAME, 'rating.clickArea')))
        score =  float(rating_area.find_element(By.CLASS_NAME,'num').text)
        num = int(rating_area.find_element(By.CLASS_NAME, 'numberofscore').text.replace('건',''))
        num_review = int(rating_area.find_element(By.CLASS_NAME, 'review').text.replace('리뷰','').replace(',',''))
        return score, num, num_review
        
    except: return None, None, None
    
def get_open_hour(cafe):
    try:
        return cafe.find_element(By.CLASS_NAME, 'openhour').text.replace('영업중\n','')
    except Exception as e: 
        print(e)
        return None
def get_cafe_info(cafe):
    data = {}
    data['name'] = get_name(cafe)
    data['category'] = get_category(cafe)
    data['location'] = get_loc(cafe)
    data['link'] = get_link(cafe)
    data['rating'], data['rating_num'], data['review_num'] = get_rating_info(cafe)
    data['open_hour'] = get_open_hour(cafe)
    return data


In [4]:
def get_cafe_list(driver):  #한 페이지에 있는 카페 정보 가져오기
    cafe_datas = []
    try:
        cafes = WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.ID, 'info.search.place.list')))
        cafes = cafes.find_elements(By.CLASS_NAME, 'PlaceItem.clickArea')
        for cafe in cafes:
            cafe_datas.append(get_cafe_info(cafe))
        return cafe_datas
    except Exception as e:
        print(e)
        return None
    
def find_next_page(driver): #다음 페이지 누르기
    try:
        page_area = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'info.search.page')))
        button = page_area.find_element(By.CLASS_NAME, 'next')
    except Exception as e:
        print(e)
        button = None
    try:
        pages = page_area.find_elements(By.TAG_NAME, 'a')
        click = False
        i = 0
        while click == False:
            page = pages[i]
            active = page.get_attribute('class')
            if active == 'ACTIVE':
                if i < 4:
                    pages[i+1].click()
                    time.sleep(3)
                    click = True
                else:
                    if (button.get_attribute('class') != 'next.disabled') or button != None:
                        button.click()
                        time.sleep(3)
                        click = True
                    else: return False
            else: i += 1
        return True
    except Exception as e:
        print(e)
        return False

def write_data(write_file, datas):

    for data in datas:
        write_file = write_file.append(data, ignore_index = True)
    
    return write_file

In [5]:
def save(df,path, code, n):
    if n < len(code):
        unicode = code[n]
        try: 
            df.to_csv(path ,encoding = unicode, index=False)
            return True
        except Exception as e:
            return save(df, path, code, n+1)
    else: return False
    

In [6]:
def cafe_crawler(driver, df):
    code = ['utf-8-sig', 'utf-8','euc-kr','cp949','utf-16','utf-16-sig','utf-32','utf-32-sig']
    page_exist = True
    while page_exist:
        datas = get_cafe_list(driver)
        if datas != None: df = write_data(df, datas)
        page_exist = find_next_page(driver)
    return df

In [7]:
def total_cafe(driver, locations):
        
    cafe_info = pd.read_csv('BDM/cafe/cafe_info.csv')
    
    code = ['utf-8-sig', 'utf-8','euc-kr','cp949','utf-16','utf-16-sig','utf-32','utf-32-sig']
    for loc in tqdm(locations):
        try:
            bar = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CLASS_NAME, 'box_searchbar')))
            enter = bar.find_element(By.TAG_NAME, 'input')
            enter.clear() 
            enter.send_keys(f"{loc} 카페")
            enter.send_keys(Keys.RETURN)
            
            #WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'info.search.place.more'))).click()
            time.sleep(1)
            sort_opitons = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'info.search.place.sort')))
            sort_opitons.find_elements(By.TAG_NAME, 'li')[-1].click()
            time.sleep(1)
            
            print(f'crawling {loc}')
            cafe_info = cafe_crawler(driver, cafe_info)
            save(cafe_info ,f'BDM/cafe/cafe_info3.csv', code, 0)
        except: continue
        

In [8]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-setuid-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("disable-infobars")
#chrome_options.add_argument("--start-maximized")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
os.environ['WDM_LOG_LEVEL'] = '0'
os.environ['WDM_LOG'] = "false"
warnings.filterwarnings("ignore")
chrome_path = 'C:\\Users\\dhkim\\IMG_Classifier\\chromedriver.exe'




In [40]:
with open('BDM/cafe/동_list.pkl','rb') as f:
    동_list=pickle.load(f)

            
            

In [41]:
동_list = 동_list[115:][70:][60:]
동_list

['연희동', '내곡동', '우이동', '왕십리동', '도림동', '송중동', '삼선동']

In [9]:
driver =  get_driver(chrome_path, chrome_options, 'https://map.kakao.com/')

In [42]:
total_cafe(driver, 동_list)

  0%|          | 0/7 [00:00<?, ?it/s]

crawling 연희동
Message: element not interactable
  (Session info: chrome=108.0.5359.94)
Stacktrace:
Backtrace:
	(No symbol) [0x007EF243]
	(No symbol) [0x00777FD1]
	(No symbol) [0x0066CF10]
	(No symbol) [0x0069D003]
	(No symbol) [0x006932A6]
	(No symbol) [0x006B858C]
	(No symbol) [0x00692BFF]
	(No symbol) [0x006B8804]
	(No symbol) [0x006CC9EB]
	(No symbol) [0x006B8386]
	(No symbol) [0x0069163C]
	(No symbol) [0x0069269D]
	GetHandleVerifier [0x00A89A22+2655074]
	GetHandleVerifier [0x00A7CA24+2601828]
	GetHandleVerifier [0x00898C0A+619850]
	GetHandleVerifier [0x00897830+614768]
	(No symbol) [0x007805FC]
	(No symbol) [0x00785968]
	(No symbol) [0x00785A55]
	(No symbol) [0x0079051B]
	BaseThreadInitThunk [0x764A6939+25]
	RtlGetFullPathName_UEx [0x770B8FD2+1218]
	RtlGetFullPathName_UEx [0x770B8F9D+1165]



 14%|█▍        | 1/7 [11:57<1:11:42, 717.17s/it]

crawling 내곡동
Message: 

local variable 'page_area' referenced before assignment


 29%|██▊       | 2/7 [22:13<54:48, 657.76s/it]  

...

 71%|███████▏  | 5/7 [52:26<20:41, 620.66s/it]

crawling 송중동
Message: element not interactable
  (Session info: chrome=108.0.5359.94)
Stacktrace:
Backtrace:
	(No symbol) [0x007EF243]
	(No symbol) [0x00777FD1]
	(No symbol) [0x0066CF10]
	(No symbol) [0x0069D003]
	(No symbol) [0x006932A6]
	(No symbol) [0x006B858C]
	(No symbol) [0x00692BFF]
	(No symbol) [0x006B8804]
	(No symbol) [0x006CC9EB]
	(No symbol) [0x006B8386]
	(No symbol) [0x0069163C]
	(No symbol) [0x0069269D]
	GetHandleVerifier [0x00A89A22+2655074]
	GetHandleVerifier [0x00A7CA24+2601828]
	GetHandleVerifier [0x00898C0A+619850]
	GetHandleVerifier [0x00897830+614768]
	(No symbol) [0x007805FC]
	(No symbol) [0x00785968]
	(No symbol) [0x00785A55]
	(No symbol) [0x0079051B]
	BaseThreadInitThunk [0x764A6939+25]
	RtlGetFullPathName_UEx [0x770B8FD2+1218]
	RtlGetFullPathName_UEx [0x770B8F9D+1165]



 86%|████████▌ | 6/7 [1:04:08<10:48, 648.50s/it]

crawling 삼선동
Message: element not interactable
  (Session info: chrome=108.0.5359.94)
Stacktrace:
Backtrace:
	(No symbol) [0x007EF243]
	(No symbol) [0x00777FD1]
	(No symbol) [0x0066CF10]
	(No symbol) [0x0069D003]
	(No symbol) [0x006932A6]
	(No symbol) [0x006B858C]
	(No symbol) [0x00692BFF]
	(No symbol) [0x006B8804]
	(No symbol) [0x006CC9EB]
	(No symbol) [0x006B8386]
	(No symbol) [0x0069163C]
	(No symbol) [0x0069269D]
	GetHandleVerifier [0x00A89A22+2655074]
	GetHandleVerifier [0x00A7CA24+2601828]
	GetHandleVerifier [0x00898C0A+619850]
	GetHandleVerifier [0x00897830+614768]
	(No symbol) [0x007805FC]
	(No symbol) [0x00785968]
	(No symbol) [0x00785A55]
	(No symbol) [0x0079051B]
	BaseThreadInitThunk [0x764A6939+25]
	RtlGetFullPathName_UEx [0x770B8FD2+1218]
	RtlGetFullPathName_UEx [0x770B8F9D+1165]



100%|██████████| 7/7 [1:18:51<00:00, 675.94s/it]


In [284]:
#서울시 전국 동 얻기
driver.get('https://www.seoul.go.kr/seoul/autonomy_sub.do')
blocks = driver.find_elements(By.CLASS_NAME,'district.tabcont')
locations = set()
for block in blocks:
    location_list = block.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME,'tr')
    for location in location_list:
        name = location.text.split('(')[0].replace('제1동','동')
        name = re.sub(r'제[\d]', '', name)
        name = re.sub(r'\d가', '', name).replace('\n','')
        locations.add(name)
locations = list(locations)
동_list = []
for l in locations:
    l = re.sub(r'\d', '', l)
    동_list.append(l.replace(',','').replace(' ',''))

with open('BDM/cafe/동_list.pkl','wb') as f:
    pickle.dump(동_list,f)

In [68]:
cafe_list = pd.read_csv('BDM/cafe/cafe_info3.csv')
cafe_list.rating_num = cafe_list.rating_num.fillna(0)
cafe_list.review_num = cafe_list.review_num.fillna(0)
cafe_list.drop_duplicates(inplace =True)
cafe_list.reset_index(inplace = True)
cafe_list.drop('index',axis = 1)

,name,category,location,rating,rating_num,review_num,open_hour,link
0,산과산사이커피로드,카페,서울 도봉구 도봉로180가길 96 4층,4.8,13.0,8.0,매일 09:00 ~ 23:00,https://place.map.kakao.com/19530655
1,카페152길,디저트카페,서울 도봉구 도봉로152길 29-3 1층,3.9,16.0,20.0,매일 08:30 ~ 22:00,https://place.map.kakao.com/1985501177
2,도봉산커피,커피전문점,서울 도봉구 도봉산4가길 4,4.7,13.0,11.0,매일 08:00 ~ 22:00,https://place.map.kakao.com/2002605651
3,로로옴,카페,서울 도봉구 도봉로152가길 152,4.5,15.0,19.0,월~금 11:00 ~ 20:00,https://place.map.kakao.com/1246426011
4,만남의광장,카페,서울 도봉구 도봉동 288-20,1.8,4.0,0.0,NaN,https://place.map.kakao.com/21927608
...,...,...,...,...,...,...,...,...
29379,케렌시아커피,카페,"서울 성북구 동소문로20나길 47 코아루센타시아 1층 108,109호",5.0,5.0,1.0,매일 11:00 ~ 22:00,https://place.map.kakao.com/1627266541
29380,미루카페,카페,서울 성북구 동소문로20길 28-3,2.5,2.0,1.0,NaN,https://place.map.kakao.com/21542542
29381,빛의 화실,커피전문점,서울 성북구 동소문로 92,5.0,1.0,2.0,NaN,https://place.map.kakao.com/1324536606
29382,헤븐드레스카페,테마카페,서울 성북구 보문로34길 35 지하1층,0.0,0.0,3.0,금일영업마감\n영업시간 수 11:00 ~ 19:00,https://place.map.kakao.com/1918079162


In [95]:
cafe_list = pd.read_csv('BDM/cafe/cafe_info.csv')
cafe_list.rating_num = cafe_list.rating_num.fillna(0)
cafe_list.review_num = cafe_list.review_num.fillna(0)
cafe_list.drop_duplicates(inplace =True)
cafe_list.reset_index(inplace = True)
cafe_list.drop('index',axis = 1)

,name,category,location,rating,rating_num,review_num,open_hour,link
0,산과산사이커피로드,카페,서울 도봉구 도봉로180가길 96 4층,4.8,13.0,8.0,매일 09:00 ~ 23:00,https://place.map.kakao.com/19530655
1,카페152길,디저트카페,서울 도봉구 도봉로152길 29-3 1층,3.9,16.0,20.0,매일 08:30 ~ 22:00,https://place.map.kakao.com/1985501177
2,도봉산커피,커피전문점,서울 도봉구 도봉산4가길 4,4.7,13.0,11.0,매일 08:00 ~ 22:00,https://place.map.kakao.com/2002605651
3,로로옴,카페,서울 도봉구 도봉로152가길 152,4.5,15.0,19.0,월~금 11:00 ~ 20:00,https://place.map.kakao.com/1246426011
4,만남의광장,카페,서울 도봉구 도봉동 288-20,1.8,4.0,0.0,NaN,https://place.map.kakao.com/21927608
...,...,...,...,...,...,...,...,...
28023,하늘아래작은마을,한식,서울 강북구 삼양로181길 253,3.5,6.0,38.0,금일영업마감\n영업시간 매일 11:00 ~ 23:00,https://place.map.kakao.com/17809096
28024,브레쉬에비뉴 방학점,"제과,베이커리","서울 도봉구 방학로 216 1,2층",3.5,17.0,7.0,금일영업마감\n영업시간 매일 08:00 ~ 23:00,https://place.map.kakao.com/24545314
28025,와플대학 방학역캠퍼스,"제과,베이커리",서울 도봉구 도봉로149길 12 아델하임 1층,5.0,5.0,16.0,금일영업마감\n영업시간 매일 10:00 ~ 22:50,https://place.map.kakao.com/1171325851
28026,파라스파라 서울 테라스카페,칵테일바,서울 강북구 삼양로 689 114동 6층,0.0,0.0,3.0,금일영업마감\n영업시간 매일 12:00 ~ 20:00,https://place.map.kakao.com/308287059


In [102]:
result = []
visited = set(cafe_list.link)
for i in range(len(cafe_list1)):
    if cafe_list1.link[i] not in visited : result.append(i)

In [101]:
cafe_list1

'https://place.map.kakao.com/1985501177'

In [103]:
cafe_list4 = cafe_list1.iloc[result, :]

In [105]:
cafe_list4.to_csv('cafe_list_left.csv', index = False)

In [82]:
cafe_list

,index,name,category,location,rating,rating_num,review_num,open_hour,link
0,0,산과산사이커피로드,카페,서울 도봉구 도봉로180가길 96 4층,4.8,13.0,8.0,매일 09:00 ~ 23:00,https://place.map.kakao.com/19530655
1,1,카페152길,디저트카페,서울 도봉구 도봉로152길 29-3 1층,3.9,16.0,20.0,매일 08:30 ~ 22:00,https://place.map.kakao.com/1985501177
2,2,도봉산커피,커피전문점,서울 도봉구 도봉산4가길 4,4.7,13.0,11.0,매일 08:00 ~ 22:00,https://place.map.kakao.com/2002605651
3,3,로로옴,카페,서울 도봉구 도봉로152가길 152,4.5,15.0,19.0,월~금 11:00 ~ 20:00,https://place.map.kakao.com/1246426011
4,4,만남의광장,카페,서울 도봉구 도봉동 288-20,1.8,4.0,0.0,NaN,https://place.map.kakao.com/21927608
...,...,...,...,...,...,...,...,...,...
29379,31631,케렌시아커피,카페,"서울 성북구 동소문로20나길 47 코아루센타시아 1층 108,109호",5.0,5.0,1.0,매일 11:00 ~ 22:00,https://place.map.kakao.com/1627266541
29380,31632,미루카페,카페,서울 성북구 동소문로20길 28-3,2.5,2.0,1.0,NaN,https://place.map.kakao.com/21542542
29381,31639,빛의 화실,커피전문점,서울 성북구 동소문로 92,5.0,1.0,2.0,NaN,https://place.map.kakao.com/1324536606
29382,31640,헤븐드레스카페,테마카페,서울 성북구 보문로34길 35 지하1층,0.0,0.0,3.0,금일영업마감\n영업시간 수 11:00 ~ 19:00,https://place.map.kakao.com/1918079162


In [22]:
def find_review_areas(driver):
    try:
        return WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "reviews_holder")))
    except: return None
    
def get_review_text(review):
    try:
        title = review.find_element(By.CLASS_NAME,'text-lg.bold').text
        text = review.find_element(By.CLASS_NAME,'mt-1').text
        return title +'.' + text
    except: return ''

def get_reviews(review_area):
    result = ''
    try:
        reviews = review_area.find_elements(By.TAG_NAME, 'article')
        for review in reviews:
            result += get_review_text(review)
        return result
    except: return None

In [36]:
def find_statement_areas(driver):
    try:
        return WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "statements_holder")))
    except: return None
    
def get_statement_text(statement):
    try: return  statement.find_element(By.CLASS_NAME,'statement_text_text.pt-2.pl-2.pr-2').text
    except: return ''

def get_statements(statement_area):
    result = ''
    try:
        statements = statement_area.find_elements(By.CLASS_NAME, 'statement')
        for statement in statements:
            result += get_statement_text(statement)
        return result
    except: return None

In [26]:
area = find_statement_areas(driver)

In [37]:
get_statements(area)

'Will always have a backup bottle. Dark, sexy and delicious. Comes in a package with a playful mood, flirt and God knows what else...;)\n68\nBottle\n8\nSillage\n9\nLongevity\n9.5\nScent\nResins and Vanilla... hard to beat if proposed by Tom Ford!\n58\nBottle\n8\nSillage\n8\nLongevity\n9\nScent\nThe warm, alluring (and unisex) Noir pour Femme by Tom Ford smells like a Brazilian caramelized condensed milk sweet called Canjica!\n44\nBottle\n6\nSillage\n7\nLongevity\n8.5\nScent\nThe opening is a bit difficult to get your nose around, but the mid and dry down make it very very well worth it. A very seductive scent.\n37\nBottle\n7\nSillage\n8\nLongevity\n8\nScent\nOriental, resinous , "black " inside too but with a gentle flowery touch. A winter or night scent with a quite good sillage.I love it\n3'